# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print("number of rows:", len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


number of rows: 186


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

167


# Part II. Complete the Apache Cassandra coding portion of your project. 

#### Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
try:
    session.execute("""CREATE KEYSPACE IF NOT EXISTS sparkifydb
                    WITH REPLICATION = {'class':'SimpleStrategy', 'replication_factor':1}""")
except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('sparkifydb')
except Exception as e:
    print(e)

## Create queries to ask the following three questions of the data

##### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession = 4

##### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    
##### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [9]:
from tqdm import tqdm # for progress bars
file = 'event_datafile_new.csv'

In [10]:
# Give me the artist, song title and song's length in the music app history that was heard during sessionId = ..., and itemInSession = ...
query1 = "SELECT artist, song, length FROM sessions WHERE sessionId=%s AND itemInSession=%s"

In [11]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below

session.execute("""CREATE TABLE IF NOT EXISTS sessions 
                   (sessionId int, 
                   itemInSession int,
                   artist text, 
                   song text,
                   length float, 
                   PRIMARY KEY (sessionId, itemInSession))
                """)

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in tqdm(csvreader):
        query = "INSERT INTO sessions (sessionId, itemInSession, artist, song, length) "
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

166it [00:00, 445.96it/s]


#### Do a SELECT to verify that the data have been inserted into each table

In [12]:
print(query1) # verify that the arguments are inserted in the correct order

SELECT artist, song, length FROM sessions WHERE sessionId=%s AND itemInSession=%s


In [13]:

try:
    rows = session.execute(query1, (112, 1)) # no results for sessionId = 338, itemInSession = 4
except Exception as e:
    print(e)
    
for row in rows:
    print("\t".join([f"{t}: {row.__getattribute__(t):25}" for t in rows.column_names]))

artist: Gershon Kingsley         	song: Pop Corn                 	length:        146.57261657714844


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = ..., sessionid = ...

In [14]:
## Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = ..., sessionid = ...
query2 = "SELECT artist, song, firstName, lastName  FROM user_sessions WHERE userId=%s AND sessionId=%s"   

In [15]:
# This table requires a composite key including itemInSession, 
# because a user might listen to a song twice during a single session.
# itemInSession also defines the ordering of the items within a user's session.
session.execute("""CREATE TABLE IF NOT EXISTS user_sessions 
                   (sessionId int, 
                   itemInSession int, 
                   artist text, 
                   song text,
                   userId int, 
                   firstName text, 
                   lastName text, 
                   PRIMARY KEY (userId, sessionId, itemInSession))
                """)

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in tqdm(csvreader):
        query = "INSERT INTO user_sessions (sessionId, itemInSession, artist, song, userId, firstName, lastName) "
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
               
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], int(line[10]), line[1], line[4]))


166it [00:00, 530.96it/s]


In [16]:
print(query2) # verify that the arguments are inserted in the correct order

SELECT artist, song, firstName, lastName  FROM user_sessions WHERE userId=%s AND sessionId=%s


In [17]:
try:
    rows = session.execute(query2, (10, 182))
except Exception as e:
    print(e)
    
for row in rows:
    print("\t".join([f"{t}: {row.__getattribute__(t):25}" for t in rows.column_names]))

artist: Down To The Bone         	song: Keep On Keepin' On       	firstname: Sylvie                   	lastname: Cruz                     
artist: Three Drives             	song: Greece 2000              	firstname: Sylvie                   	lastname: Cruz                     
artist: Sebastien Tellier        	song: Kilometer                	firstname: Sylvie                   	lastname: Cruz                     
artist: Lonnie Gordon            	song: Catch You Baby (Steve Pitron & Max Sanna Radio Edit)	firstname: Sylvie                   	lastname: Cruz                     


### 3. Give me every user name (first and last) in my music app history who listened to the song '...'

In [18]:
query3 = "SELECT firstName, lastName FROM song_users WHERE song=%s"

In [19]:
# This table requires a composite key because multiple people may listen to the same song.
# Combining the song name with the sessionId and the itemInSession number results in a unique composite key.
session.execute("""CREATE TABLE IF NOT EXISTS song_users 
                   (sessionId int, 
                   itemInSession int,
                   song text, 
                   firstName text, 
                   lastName text, 
                   PRIMARY KEY (song, sessionId, itemInSession))
                """)

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in tqdm(csvreader):
        query = "INSERT INTO song_users (sessionId, itemInSession, song, firstName, lastName) "
        query = query + "VALUES (%s, %s, %s, %s, %s)"
               
        session.execute(query, (int(line[8]), int(line[3]), line[9], line[1], line[4]))

166it [00:00, 510.03it/s]


In [20]:
# no results for 'All Hands Against His Own', so I verify with 'Eriatarka'
try:
    rows = session.execute(query3, ('Eriatarka',)) 
except Exception as e:
    print(e)
    
for row in rows:
    print("\t".join([f"{t}: {row.__getattribute__(t):15}" for t in rows.column_names]))

firstname: Kaylee         	lastname: Summers        


### Drop the tables before closing out the sessions

In [21]:
session.execute("DROP TABLE sessions")
session.execute("DROP TABLE user_sessions")
session.execute("DROP TABLE song_users")

### Close the session and cluster connection¶

In [22]:
session.shutdown()
cluster.shutdown()